In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# 1. Carregar o Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
column_names_pt_br = [
    'classe', 'alcool', 'acido_malico', 'cinzas', 'alcalinidade_de_cinzas',
    'magnesio', 'fenois_totais', 'flavanoides', 'fenois_nao_flavanoides',
    'proantocianinas', 'intensidade_de_cor', 'matiz',
    'od280_od315_de_vinhos_diluidos', 'prolina'
]
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# 2. Pré-processamento dos Dados
X = vinhos.drop(columns='classe')
y = vinhos['classe']

# Função para realizar o experimento
def realizar_experimento(random_state, k_values=[3, 5]):
    resultados = {}
    for k in k_values:
        accuracies = []

        # Configuração do KFold
        kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Treinamento e avaliação do modelo KNN
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)

        # Armazenamento dos resultados
        resultados[k] = {
            'media': np.mean(accuracies),
            'desvio_padrao': np.std(accuracies)
        }

    return resultados

# 3. Execução dos Experimentos
random_states = [42, 17, 24]
resultados_finais = {}

for random_state in random_states:
    resultados_finais[random_state] = realizar_experimento(random_state)

# Exibir os resultados
for random_state, resultados in resultados_finais.items():
    print(f"\nRandom State: {random_state}")
    for k, valores in resultados.items():
        print(f"K={k}: Média da acurácia = {valores['media']:.4f}, Desvio Padrão = {valores['desvio_padrao']:.4f}")

# 4. Análise Final
for random_state, resultados in resultados_finais.items():
    melhor_k = max(resultados, key=lambda k: resultados[k]['media'])
    print(f"\nPara Random State {random_state}, o melhor valor de K é {melhor_k} com acurácia média de {resultados[melhor_k]['media']:.4f}")



Random State: 42
K=3: Média da acurácia = 0.7036, Desvio Padrão = 0.1392
K=5: Média da acurácia = 0.6641, Desvio Padrão = 0.1001

Random State: 17
K=3: Média da acurácia = 0.6987, Desvio Padrão = 0.1040
K=5: Média da acurácia = 0.6876, Desvio Padrão = 0.1121

Random State: 24
K=3: Média da acurácia = 0.7127, Desvio Padrão = 0.0968
K=5: Média da acurácia = 0.7127, Desvio Padrão = 0.0940

Para Random State 42, o melhor valor de K é 3 com acurácia média de 0.7036

Para Random State 17, o melhor valor de K é 3 com acurácia média de 0.6987

Para Random State 24, o melhor valor de K é 3 com acurácia média de 0.7127
